<a href="https://colab.research.google.com/github/kevindhuertas/market-prediction-with-transformer-model/blob/main/main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TA-Lib


  Using cached TA-Lib-0.5.2.tar.gz (372 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib)


In [ ]:
!apt-get install -y build-essential
!apt-get install -y libta-lib0-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libta-lib0-dev


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import pandas_ta as ta  # Librería para indicadores técnicos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# =============================================
# ANÁLISIS DE DATOS: APPLE, AMAZON Y OTRAS EMPRESAS
# =============================================

# ---------
# Librerías
# ---------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import pandas_ta as ta  # Librería para indicadores técnicos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Para visualizar las gráficas en el notebook
%matplotlib inline

# -----------------------------------------------------------------------
# 1. Descarga de Datos de Varias Empresas con Velas de 5 Minutos
# -----------------------------------------------------------------------
# Definimos los símbolos y el periodo
symbols = ["AAPL"]
period = "3mo"  # Periodo de 1 mes
interval = "15m"  # Intervalo de 5 minutos

# Descargamos los datos
stocks_data = {}
for symbol in symbols:
    try:
        print(f"Descargando datos para {symbol}...")
        stocks_data[symbol] = yf.download(symbol, period=period, interval=interval, progress=False)
    except Exception as e:
        print(f"Error descargando datos para {symbol}: {e}")

# Concatenamos todos los DataFrames en uno solo
df_all = pd.concat(stocks_data.values(), axis=0)

# Verificamos si se descargaron datos correctamente
if df_all.empty:
    print("No se pudieron descargar los datos. Verifique el rango de fechas y el intervalo.")
else:
    # Aseguramos que las columnas estén en un formato plano
    if isinstance(df_all.columns, pd.MultiIndex):
        df_all.columns = df_all.columns.get_level_values(0)

    # Revisamos las primeras filas del DataFrame combinado
    print("Primeras filas del DataFrame combinado:")
    display(df_all.head())

    # -----------------------------------------------------------------------
    # 2. Agregar Indicadores Técnicos con pandas_ta
    # -----------------------------------------------------------------------

    def add_technical_indicators_with_pandas_ta(df):
        df = df.copy()

        # Indicadores de tendencia
        df.ta.sma(length=50, append=True)  # Media móvil simple (50)
        df.ta.sma(length=100, append=True)  # Media móvil simple (100)
        df.ta.ema(length=50, append=True)  # Media móvil exponencial (50)
        df.ta.ema(length=100, append=True)  # Media móvil exponencial (100)

        # Indicadores de momentum
        df.ta.rsi(length=14, append=True)  # Relative Strength Index (14)
        df.ta.macd(append=True)  # Moving Average Convergence Divergence
        df.ta.stoch(append=True)  # Stochastic Oscillator (K, D)

        # Indicadores de volatilidad
        df.ta.atr(length=14, append=True)  # Average True Range (14)
        df.ta.bbands(length=20, append=True)  # Bollinger Bands (20, 2)

        # Indicadores de volumen
        df.ta.obv(append=True)  # On-Balance Volume
        df.ta.cmf(length=20, append=True)  # Chaikin Money Flow (20)

        return df

    # Agregamos los indicadores técnicos al DataFrame combinado
    df_all = add_technical_indicators_with_pandas_ta(df_all)

    print("Primeras filas del DataFrame combinado con indicadores técnicos:")
    display(df_all.head())

    # -----------------------------------------------------------------------
    # 3. Limpieza y Normalización de Datos
    # -----------------------------------------------------------------------

    # 3.1 Eliminar filas con valores nulos
    print("Valores nulos antes de la limpieza:")
    print(df_all.isnull().sum())

    df_all = df_all.dropna()

    print("Valores nulos después de la limpieza:")
    print(df_all.isnull().sum())

    # 3.2 Normalización de columnas numéricas
    scaler = MinMaxScaler()
    columns_to_normalize = [
        "Open", "High", "Low", "Close", "Volume",
        "SMA_50", "SMA_100", "EMA_50", "EMA_100",
        "RSI_14", "MACD_12_26_9", "MACDs_12_26_9",
        "BBL_20_2.0", "BBM_20_2.0", "BBU_20_2.0",
        "ATRr_14", "CMF_20"
    ]

    # Escalamos solo las columnas relevantes
    df_all[columns_to_normalize] = scaler.fit_transform(df_all[columns_to_normalize])

    # Imprimir datos después de la normalización
    print("Datos después de la normalización:")
    display(df_all[columns_to_normalize].head())

    # 3.3 Separar las características y etiquetas
    features = df_all[columns_to_normalize]
    df_all["Target"] = df_all["Close"].shift(-3) > df_all["Close"]  # Etiqueta para predicción de 3 velas
    targets = df_all["Target"]

    # 3.4 Dividir los datos en conjuntos de entrenamiento, validación y prueba
    X_train, X_temp, y_train, y_temp = train_test_split(features, targets, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    print("Tamaños de los conjuntos:")
    print(f"Entrenamiento: {X_train.shape}, Validación: {X_val.shape}, Prueba: {X_test.shape}")

# -----------------------------------------------------------------------
# Observaciones
# -----------------------------------------------------------------------
# - Se utilizan velas de 5 minutos para el análisis con un periodo de 1 mes.
# - Los datos de múltiples empresas se combinan en un único DataFrame sin columnas adicionales.
# - Se eliminaron las filas con valores nulos.
# - Las características fueron normalizadas al rango [0, 1].
# - Los datos fueron divididos en conjuntos de entrenamiento, validación y prueba.


Descargando datos para AAPL...


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=3mo) (Yahoo error = "15m data not available for startTime=1727309085 and endTime=1735344285. The requested range must be within the last 60 days.")')


No se pudieron descargar los datos. Verifique el rango de fechas y el intervalo.
